# Text classification from scratch

**Authors:** Mark Omernick, Francois Chollet<br>
**Date created:** 2019/11/06<br>
**Last modified:** 2020/05/17<br>
**Description:** Text sentiment classification starting from raw text files.

## Introduction

This example shows how to do text classification starting from raw text (as
a set of text files on disk). We demonstrate the workflow on the IMDB sentiment
classification dataset (unprocessed version). We use the `TextVectorization` layer for
 word splitting & indexing.

## Setup

In [1]:
import tensorflow as tf
import numpy as np
import os
from keras.utils import np_utils
from tensorflow.keras.layers import Embedding, Conv1D, AveragePooling1D, Bidirectional, LSTM, Dense

## Load the data: IMDB movie review sentiment classification

Let's download the data and inspect its structure.

In [2]:
!curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xf aclImdb_v1.tar.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 80.2M  100 80.2M    0     0  7317k      0  0:00:11  0:00:11 --:--:--  9.8M


The `aclImdb` folder contains a `train` and `test` subfolder:

In [3]:
!ls aclImdb

imdbEr.txt  imdb.vocab	README	test  train


In [4]:
!ls aclImdb/test

labeledBow.feat  neg  pos  urls_neg.txt  urls_pos.txt


In [5]:
!ls aclImdb/train

labeledBow.feat  pos	unsupBow.feat  urls_pos.txt
neg		 unsup	urls_neg.txt   urls_unsup.txt


The `aclImdb/train/pos` and `aclImdb/train/neg` folders contain text files, each of
 which represents one review (either positive or negative):

In [6]:
!cat aclImdb/train/pos/6248_7.txt

Being an Austrian myself this has been a straight knock in my face. Fortunately I don't live nowhere near the place where this movie takes place but unfortunately it portrays everything that the rest of Austria hates about Viennese people (or people close to that region). And it is very easy to read that this is exactly the directors intention: to let your head sink into your hands and say "Oh my god, how can THAT be possible!". No, not with me, the (in my opinion) totally exaggerated uncensored swinger club scene is not necessary, I watch porn, sure, but in this context I was rather disgusted than put in the right context.<br /><br />This movie tells a story about how misled people who suffer from lack of education or bad company try to survive and live in a world of redundancy and boring horizons. A girl who is treated like a whore by her super-jealous boyfriend (and still keeps coming back), a female teacher who discovers her masochism by putting the life of her super-cruel "lover" 

We are only interested in the `pos` and `neg` subfolders, so let's delete the rest:

In [7]:
!rm -r aclImdb/train/unsup

In [8]:
for dir in range (8):
    os.mkdir(f'aclImdb/train/{dir}')

In [9]:
for dir in range (8):
    os.mkdir(f'aclImdb/test/{dir}')
!ls aclImdb/test 

0  1  2  3  4  5  6  7	labeledBow.feat  neg  pos  urls_neg.txt  urls_pos.txt


You can use the utility `tf.keras.utils.text_dataset_from_directory` to
generate a labeled `tf.data.Dataset` object from a set of text files on disk filed
 into class-specific folders.

Let's use it to generate the training, validation, and test datasets. The validation
and training datasets are generated from two subsets of the `train` directory, with 20%
of samples going to the validation dataset and 80% going to the training dataset.

Having a validation dataset in addition to the test dataset is useful for tuning
hyperparameters, such as the model architecture, for which the test dataset should not
be used.

Before putting the model out into the real world however, it should be retrained using all
available training data (without creating a validation dataset), so its performance is maximized.

When using the `validation_split` & `subset` arguments, make sure to either specify a
random seed, or to pass `shuffle=False`, so that the validation & training splits you
get have no overlap.

In [10]:
directory = 'aclImdb/train'
 
for root, dirs, files in os.walk(directory):
    if root == 'aclImdb/train/neg' or root== 'aclImdb/train/pos':
        for file_ in files:
            if 1 <= int(file_[-5]) <= 4:
                dir_ = int(file_[-5])-1
            elif 7 <= int(file_[-5]) <= 9:
                dir_ = int(file_[-5])-3
            else:  
                dir_ = int(file_[-6:-4])-3

            os.rename(''.join([root,'/',file_]), f'aclImdb/train/{dir_}/{file_}')

In [11]:
!rm -r aclImdb/train/neg
!rm -r aclImdb/train/pos

In [12]:
!ls aclImdb/train 

0  2  4  6  labeledBow.feat  urls_neg.txt  urls_unsup.txt
1  3  5  7  unsupBow.feat    urls_pos.txt


In [13]:
directory = 'aclImdb/test'
 
for root, dirs, files in os.walk(directory):
    if root == 'aclImdb/test/neg' or root== 'aclImdb/test/pos':
        for file_ in files:
            if 1 <= int(file_[-5]) <= 4:
                dir_ = int(file_[-5])-1
            elif 7 <= int(file_[-5]) <= 9:
                dir_ = int(file_[-5])-3
            else:  
                dir_ = int(file_[-6:-4])-3

            os.rename(''.join([root,'/',file_]), f'aclImdb/test/{dir_}/{file_}')

!rm -r aclImdb/test/neg
!rm -r aclImdb/test/pos

In [14]:
!ls aclImdb/test

0  1  2  3  4  5  6  7	labeledBow.feat  urls_neg.txt  urls_pos.txt


In [47]:
batch_size = 32
raw_train_ds = tf.keras.utils.text_dataset_from_directory(
    "aclImdb/train",
    batch_size=batch_size,
    validation_split=0.2,
    subset="training",
    seed=1337,
)
raw_val_ds = tf.keras.utils.text_dataset_from_directory(
    "aclImdb/train",
    batch_size=batch_size,
    validation_split=0.2,
    subset="validation",
    seed=1337,
)
raw_test_ds = tf.keras.utils.text_dataset_from_directory(
    "aclImdb/test", batch_size=batch_size)

print(f"Number of batches in raw_train_ds: {raw_train_ds.cardinality()}")
print(f"Number of batches in raw_val_ds: {raw_val_ds.cardinality()}")
print(f"Number of batches in raw_test_ds: {raw_test_ds.cardinality()}")

Found 25000 files belonging to 8 classes.
Using 20000 files for training.
Found 25000 files belonging to 8 classes.
Using 5000 files for validation.
Found 25000 files belonging to 8 classes.
Number of batches in raw_train_ds: 625
Number of batches in raw_val_ds: 157
Number of batches in raw_test_ds: 782


In [16]:
for i, label in enumerate(raw_train_ds.class_names):
  print("Label", i, "corresponds to", label)

Label 0 corresponds to 0
Label 1 corresponds to 1
Label 2 corresponds to 2
Label 3 corresponds to 3
Label 4 corresponds to 4
Label 5 corresponds to 5
Label 6 corresponds to 6
Label 7 corresponds to 7


Let's preview a few samples:

In [17]:
for comments, labels in raw_train_ds.take(1):
    for i in range(5):
        print(comments.numpy()[i])
        print(labels.numpy()[i])

b'I watched it subtitled as it was in Russian, but really enjoyed it. The main character Sasha was born cursed, with a deadly weapon as an extension of his body. He lived his whole life unhappy because he was different and because anger caused him to do deadly things. <br /><br />When Sasha finally found love in a young woman named Katya everyone tried to take her away from him ending in a deadly battle. There was a fair amount of gore, but not too much for the weak stomached.<br /><br />Not for people who like the regular old Hollywood movie, but for those who enjoy independent films. Kinda got the feeling of an Asian fantasy film.'
5
b'I have this movie on DVD and must have watched it thirty times by now. I must really love it, right? Well, not really.<br /><br />I was a surfer earlier in my life, and I loved the sport. To this day, I am fascinated by good surfing. Riding Giants has plenty of that, and thus I am a sucker for the thing. But I definitely have some bones to pick with it

## Prepare the data

In particular, we remove `<br />` tags.

In [29]:
from tensorflow.keras.layers import TextVectorization
import string
import re

# Having looked at our data above, we see that the raw text contains HTML break
# tags of the form '<br />'. These tags will not be removed by the default
# standardizer (which doesn't strip HTML). Because of this, we will need to
# create a custom standardization function.
def custom_standardization(input_data):
    lowercase = tf.strings.lower(input_data)
    stripped_html = tf.strings.regex_replace(lowercase, "<br />", " ")
    return tf.strings.regex_replace(
        stripped_html, f"[{re.escape(string.punctuation)}]", ""
    )


# Model constants.
max_features = 20000
embedding_dim = 128
sequence_length = 500

# Now that we have our custom standardization, we can instantiate our text
# vectorization layer. We are using this layer to normalize, split, and map
# strings to integers, so we set our 'output_mode' to 'int'.
# Note that we're using the default split function,
# and the custom standardization defined above.
# We also set an explicit maximum sequence length, since the CNNs later in our
# model won't support ragged sequences.
vectorize_layer = TextVectorization(
    standardize=custom_standardization,
    max_tokens=max_features,
    output_mode="int",
    output_sequence_length=sequence_length,
)

# Now that the vocab layer has been created, call `adapt` on a text-only
# dataset to create the vocabulary. You don't have to batch, but for very large
# datasets this means you're not keeping spare copies of the dataset in memory.

# Let's make a text-only dataset (no labels):
text_ds = raw_train_ds.map(lambda x, y: x)
# Let's call `adapt`:
vectorize_layer.adapt(text_ds)

## Two options to vectorize the data

There are 2 ways we can use our text vectorization layer:

**Option 1: Make it part of the model**, so as to obtain a model that processes raw
 strings, like this:

```python
text_input = tf.keras.Input(shape=(1,), dtype=tf.string, name='text')
x = vectorize_layer(text_input)
x = layers.Embedding(max_features + 1, embedding_dim)(x)
...
```

**Option 2: Apply it to the text dataset** to obtain a dataset of word indices, then
 feed it into a model that expects integer sequences as inputs.

An important difference between the two is that option 2 enables you to do
**asynchronous CPU processing and buffering** of your data when training on GPU.
So if you're training the model on GPU, you probably want to go with this option to get
 the best performance. This is what we will do below.

If we were to export our model to production, we'd ship a model that accepts raw
strings as input, like in the code snippet for option 1 above. This can be done after
 training. We do this in the last section.


In [31]:

def vectorize_text(text, label):
    text = tf.expand_dims(text, -1)
    return vectorize_layer(text), label


# Vectorize the data.
train_ds = raw_train_ds.map(vectorize_text)
val_ds = raw_val_ds.map(vectorize_text)
test_ds = raw_test_ds.map(vectorize_text)

# Do async prefetching / buffering of the data for best performance on GPU.
train_ds = train_ds.cache().prefetch(buffer_size=10)
val_ds = val_ds.cache().prefetch(buffer_size=10)
test_ds = test_ds.cache().prefetch(buffer_size=10)

## Build a model

We choose a simple 1D convnet starting with an `Embedding` layer.

In [32]:
from tensorflow.keras import layers

In [23]:
# A integer input for vocab indices.
inputs = tf.keras.Input(shape=(None,), dtype="int64")

# Next, we add a layer to map those vocab indices into a space of dimensionality
# 'embedding_dim'.
x = layers.Embedding(max_features, embedding_dim)(inputs)
#x = layers.Dropout(0.5)(x)

# Conv1D + global max pooling
# x = layers.Conv1D(128, 7, padding="valid", activation="relu", strides=3)(x)
# x = layers.Conv1D(128, 7, padding="valid", activation="relu", strides=3)(x)
x = layers.LSTM(128, return_sequences=True, return_state=False)(x)
x = layers.LSTM(256, return_sequences=True, return_state=False)(x)
x = layers.GlobalMaxPooling1D()(x)

# We add a vanilla hidden layer:
x = layers.Dense(128, activation="relu")(x)
#x = layers.Dropout(0.5)(x)

# We project onto a single unit output layer, and squash it with a sigmoid:
predictions = tf.keras.layers.Dense(8, activation="softmax", name="predictions")(x)

model = tf.keras.Model(inputs, predictions)

# Compile the model with binary crossentropy loss and an adam optimizer.
model.compile(loss='sparse_categorical_crossentropy', optimizer="adam", metrics=["accuracy"])

In [24]:
# A integer input for vocab indices.
inputs = tf.keras.Input(shape=(None,), dtype="int64")

# Next, we add a layer to map those vocab indices into a space of dimensionality
# 'embedding_dim'.
x = layers.Embedding(max_features, embedding_dim)(inputs)
x = layers.Conv1D(filters = 32, kernel_size = 3, padding = 'same', activation = 'relu')(x)
x = layers.AveragePooling1D(pool_size = 2)(x)
x = layers.Bidirectional(LSTM(200, dropout = 0.5))(x)
# We project onto a single unit output layer, and squash it with a sigmoid:
predictions = tf.keras.layers.Dense(8, activation="softmax", name="predictions")(x)

model = tf.keras.Model(inputs, predictions)

# Compile the model with binary crossentropy loss and an adam optimizer.
model.compile(loss='sparse_categorical_crossentropy', optimizer="adam", metrics=["accuracy"])

In [25]:
inputs = tf.keras.Input(shape=(None,), dtype="int64")
###x = layers.Embedding(129892, 32)(inputs)
x = layers.Embedding(max_features, embedding_dim)(inputs)
x = layers.Bidirectional(LSTM(75, dropout = 0.1))(x)
predictions = tf.keras.layers.Dense(8, activation="softmax", name="predictions")(x)

model = tf.keras.Model(inputs, predictions)

# Compile the model with binary crossentropy loss and an adam optimizer.
model.compile(loss='sparse_categorical_crossentropy', optimizer="adam", metrics=["accuracy"])

inputs = tf.keras.Input(shape=(None,), dtype="int64")

x = layers.Embedding(max_features, embedding_dim)(inputs)
x = layers.Bidirectional(layers.LSTM(64, return_sequences=True))(x)
x = layers.Bidirectional(layers.LSTM(64))(x)
predictions = tf.keras.layers.Dense(10, activation="softmax", name="predictions")(x)

model = tf.keras.Model(inputs, predictions)

# Compile the model with binary crossentropy loss and an adam optimizer.
model.compile(loss='sparse_categorical_crossentropy', optimizer="adam", metrics=["accuracy"])

## Train the model

In [ ]:
epochs = 10

# Fit the model using the train and test datasets.
model.fit(train_ds, validation_data=val_ds, epochs=epochs)

Epoch 1/10
625/625 [==============================] - 744s 1s/step - loss: 1.9022 - accuracy: 0.2865 - val_loss: 1.7429 - val_accuracy: 0.3446
Epoch 2/10
625/625 [==============================] - 784s 1s/step - loss: 1.6524 - accuracy: 0.3776 - val_loss: 1.9581 - val_accuracy: 0.2474
Epoch 3/10
  3/625 [..............................] - ETA: 10:30 - loss: 1.9202 - accuracy: 0.3750

## Evaluate the model on the test set

In [ ]:
model.evaluate(test_ds)

## Make an end-to-end model

If you want to obtain a model capable of processing raw strings, you can simply
create a new model (using the weights we just trained):

In [ ]:
# A string input
inputs = tf.keras.Input(shape=(1,), dtype="string")
# Turn strings into vocab indices
indices = vectorize_layer(inputs)
# Turn vocab indices into predictions
outputs = model(indices)

# Our end to end model
end_to_end_model = tf.keras.Model(inputs, outputs)
end_to_end_model.compile(
    loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"]
)

# Test it with `raw_test_ds`, which yields raw strings
end_to_end_model.evaluate(raw_test_ds)

#BERT

In [18]:
!pip install -q -U "tensorflow-text==2.11.*"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 87.6 MB/s eta 0:00:00


In [19]:
!pip install -q tf-models-official==2.11.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 52.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 KB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 KB 5.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 238.9/238.9 KB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 352.1/352.1 KB 31.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 662.4/662.4 KB 31.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 55.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.2/38.2 MB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 50.4 MB/s eta 0:00:00


In [20]:
import os
import shutil

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from official.nlp import optimization  

import matplotlib.pyplot as plt

tf.get_logger().setLevel('ERROR')

In [21]:
bert_model_name = 'small_bert/bert_en_uncased_L-4_H-512_A-8'
tfhub_handle_encoder = 'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1'
tfhub_handle_preprocess = 'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3'

In [22]:
bert_preprocess_model = hub.KerasLayer(tfhub_handle_preprocess)

In [23]:
text_test = ['this is such an amazing movie!']
text_preprocessed = bert_preprocess_model(text_test)

print(f'Keys       : {list(text_preprocessed.keys())}')
print(f'Shape      : {text_preprocessed["input_word_ids"].shape}')
print(f'Word Ids   : {text_preprocessed["input_word_ids"][0, :12]}')
print(f'Input Mask : {text_preprocessed["input_mask"][0, :12]}')
print(f'Type Ids   : {text_preprocessed["input_type_ids"][0, :12]}')

Keys       : ['input_mask', 'input_word_ids', 'input_type_ids']
Shape      : (1, 128)
Word Ids   : [ 101 2023 2003 2107 2019 6429 3185  999  102    0    0    0]
Input Mask : [1 1 1 1 1 1 1 1 1 0 0 0]
Type Ids   : [0 0 0 0 0 0 0 0 0 0 0 0]


In [24]:
bert_model = hub.KerasLayer(tfhub_handle_encoder)

In [25]:
bert_results = bert_model(text_preprocessed)

print(f'Loaded BERT: {tfhub_handle_encoder}')
print(f'Pooled Outputs Shape:{bert_results["pooled_output"].shape}')
print(f'Pooled Outputs Values:{bert_results["pooled_output"][0, :12]}')
print(f'Sequence Outputs Shape:{bert_results["sequence_output"].shape}')
print(f'Sequence Outputs Values:{bert_results["sequence_output"][0, :12]}')

Loaded BERT: https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1
Pooled Outputs Shape:(1, 512)
Pooled Outputs Values:[ 0.76262873  0.9928099  -0.18611872  0.36673853  0.1523371   0.65504426
  0.9681154  -0.94862705  0.00216182 -0.9877732   0.06842728 -0.97630584]
Sequence Outputs Shape:(1, 128, 512)
Sequence Outputs Values:[[-0.28946307  0.34321266  0.33231518 ...  0.21300897  0.71020764
  -0.05771176]
 [-0.2874206   0.3198098  -0.23018597 ...  0.5845501  -0.21329726
   0.72692066]
 [-0.6615712   0.68876785 -0.8743292  ...  0.10877268 -0.2617322
   0.4785534 ]
 ...
 [-0.22561154 -0.28925663 -0.07064398 ...  0.47566003  0.8327717
   0.40025344]
 [-0.29824272 -0.27473173 -0.05450511 ...  0.48849759  1.0955356
   0.18163322]
 [-0.44378266  0.00930682  0.07223728 ...  0.17290092  1.1833241
   0.07898061]]


In [63]:
def build_classifier_model():
  text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
  preprocessing_layer = hub.KerasLayer(tfhub_handle_preprocess, name='preprocessing')
  encoder_inputs = preprocessing_layer(text_input)

  encoder = hub.KerasLayer(tfhub_handle_encoder, trainable=True, name='BERT_encoder')
  outputs = encoder(encoder_inputs)
  net = outputs['pooled_output']
  net = tf.keras.layers.Dropout(0.1)(net)
  net = tf.keras.layers.Dense(8, activation="softmax", name='classifier')(net)

  return tf.keras.Model(text_input, net)

In [64]:
classifier_model = build_classifier_model()
bert_raw_result = classifier_model(tf.constant(text_test))
print(bert_raw_result)

tf.Tensor(
[[0.02913774 0.01134778 0.01559757 0.56673855 0.17262267 0.02213632
  0.04480406 0.13761538]], shape=(1, 8), dtype=float32)


In [42]:
tf.keras.utils.plot_model(classifier_model)

In [65]:
loss = tf.keras.losses.SparseCategoricalCrossentropy()
metrics = tf.metrics.Accuracy()

In [66]:
epochs = 5
steps_per_epoch = tf.data.experimental.cardinality(train_ds).numpy()
num_train_steps = steps_per_epoch * epochs
num_warmup_steps = int(0.1*num_train_steps)

init_lr = 3e-5
optimizer = optimization.create_optimizer(init_lr=init_lr,
                                          num_train_steps=num_train_steps,
                                          num_warmup_steps=num_warmup_steps,
                                          optimizer_type='adamw')

In [69]:
classifier_model.compile(optimizer=optimizer,
                         loss='sparse_categorical_crossentropy',
                         metrics=["accuracy"])

In [70]:
history = classifier_model.fit(raw_train_ds,
                               validation_data=raw_val_ds,
                               epochs=epochs)

Epoch 1/5
625/625 [==============================] - 177s 278ms/step - loss: 1.8351 - accuracy: 0.3092 - val_loss: 1.6220 - val_accuracy: 0.3800
Epoch 2/5
625/625 [==============================] - 170s 273ms/step - loss: 1.5556 - accuracy: 0.3968 - val_loss: 1.5794 - val_accuracy: 0.3982
Epoch 3/5
625/625 [==============================] - 173s 277ms/step - loss: 1.4345 - accuracy: 0.4414 - val_loss: 1.5900 - val_accuracy: 0.3990
Epoch 4/5
625/625 [==============================] - 169s 270ms/step - loss: 1.3263 - accuracy: 0.4830 - val_loss: 1.6098 - val_accuracy: 0.4018
Epoch 5/5
625/625 [==============================] - 169s 271ms/step - loss: 1.2544 - accuracy: 0.5131 - val_loss: 1.6181 - val_accuracy: 0.3994


In [72]:
loss, accuracy = classifier_model.evaluate(raw_test_ds)

print(f'Loss: {loss}')
print(f'Accuracy: {accuracy}')

782/782 [==============================] - 101s 128ms/step - loss: 1.6138 - accuracy: 0.3978
Loss: 1.6138099431991577
Accuracy: 0.3978399932384491
